In [1]:
from ultralytics import YOLO
from pathlib import Path
import torch
import cv2
import time
import numpy as np
from shapely.geometry import Polygon
import asyncio

In [2]:
# Try to load model
if Path("yolov8s.pt").exists():
	model = YOLO("yolov8s.pt")

else:
	# Build from YAML and transfer weights
	model = YOLO("yolov8s.yaml").load("yolov8s.pt")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_s

In [3]:
def onMouseClick(event, x, y, flags, param):
	global points

	if event == cv2.EVENT_LBUTTONDOWN:
		print(x, y)
		points.append((x, y))

def check_overlap(boxes, polygon_points):
	# Not enough points to form a polygon
	if len(polygon_points) < 3:
		return False
	
	user_polygon = np.array(polygon_points, np.int32)
	user_poly_shape = Polygon(user_polygon)
	
	for box in boxes:
		box_polygon = np.array([
			(box[0], box[1]),
			(box[2], box[1]),
			(box[2], box[3]),
			(box[0], box[3]),
		], np.int32)
		
		if Polygon(box_polygon).intersects(user_poly_shape):
			return True
	
	return False

async def process_frame(frame):
	small_frame = cv2.resize(frame, (640, 480))
	results = await asyncio.to_thread(model, source=small_frame, conf=0.5, save=False, classes=[0], verbose=False)
	return results[0]


In [4]:
async def main():
	global points

	source = cv2.VideoCapture(0)
	cv2.namedWindow("YOLO Output")
	cv2.setMouseCallback("YOLO Output", onMouseClick)

	# Points of zone
	points = []

	# For FPS calculation
	new_frame_time = 0
	prev_frame_time = 0

	# Frame skipper
	frame_skip = 2
	frame_count = 0

	while True:
		ret, frame = source.read()
		# Detect objects
		if frame_count % frame_skip == 0:
			results = await process_frame(frame)
			frame_count = 0
		
		frame_count += 1
		frame = results.plot()

		# Add circles at point coords
		for i, point in enumerate(points):
			# Draw circle at point
			cv2.circle(frame, point, 5, (243, 211, 74), 2) 
			# Draw line to connect to next point
			if len(points) > 1:
				next_point = points[i+1] if i < len(points)-1 else points[0]
				cv2.line(frame, point, next_point, (243, 211, 74), 2)

		if check_overlap(results.boxes.xyxy.tolist(), points):
			cv2.putText(frame, "Zone overlap", (7, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

		# font which we will be using to display FPS 
		font = cv2.FONT_HERSHEY_SIMPLEX 
		# time when we finish processing for this frame 
		new_frame_time = time.time()
	
		# Calculate FPS6
		fps = 1 / (new_frame_time-prev_frame_time) 
		prev_frame_time = new_frame_time 
		fps = str(int(fps))
		cv2.putText(frame, fps, (100, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)
		
		frame = cv2.resize(frame, (frame.shape[1], frame.shape[0]))
		cv2.imshow("YOLO Output", frame)
		
		if cv2.waitKey(10) & 0xFF == ord("q"):
			break

	source.release()
	cv2.destroyAllWindows()
	
await main()

345 63
511 69
503 186
327 175
